# Applied Quantum Computing Lab: Shor's Algorithm (SOLUTIONS)

This lab will guide you through implementing and understanding Shor's algorithm, one of the most significant quantum algorithms with revolutionary implications for cryptography.

## Prerequisites
- Understanding of quantum gates and quantum circuits
- Familiarity with Python and Qiskit
- Basic knowledge of modular arithmetic

## Learning Objectives
By completing this lab, you will:
1. Understand the foundational principles of Shor's algorithm
2. Implement key components of the quantum period-finding subroutine
3. Apply Shor's algorithm to factorize small numbers
4. Explore the implications of Shor's algorithm for cryptography

## Introduction

Shor's algorithm, developed by Peter Shor in 1994, provides an efficient quantum method for finding the prime factors of an integer. This algorithm is particularly significant because it can break RSA encryption, one of the most widely used public-key cryptography systems.

### Key Concepts:

1. **Problem Statement**: Given a non-prime integer N, find its prime factors.
   - Classical approach: Best known algorithms are sub-exponential in the number of digits (e.g., General Number Field Sieve)
   - Shor's approach: Polynomial time in the number of digits

2. **Quantum Advantage**: Shor's algorithm achieves this extraordinary speedup by using quantum superposition to find the period of a function, which is then used to determine the factors.

3. **Algorithm Structure**:
   - Classical part: Reframing the factoring problem as finding the period of a function
   - Quantum part: Using quantum Fourier transform to efficiently find this period
   - Classical part: Using the period to compute the factors

Let's implement Shor's algorithm step by step.

## Exercise 1: Understanding the Classical Components

Before diving into the quantum part, it's important to understand how the classical parts of Shor's algorithm work and how the factoring problem relates to period finding.

First, let's import the required libraries:

In [ ]:
# Import required libraries for Qiskit 2.0.2
import numpy as np
from math import gcd  # Greatest common divisor function
from fractions import Fraction
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT
import matplotlib.pyplot as plt

# Set a fixed seed for reproducibility
np.random.seed(42)

# Display matplotlib plots inline
%matplotlib inline

### Task 1.1: Implementing Classical Pre-processing

Let's implement the classical part of Shor's algorithm that sets up the period finding problem.

The key insight is that factoring can be reduced to finding the period of the function f(x) = a^x mod N, where:
- N is the number we want to factorize
- a is a randomly chosen number such that 1 < a < N and gcd(a, N) = 1

Complete the following functions:

In [ ]:
# Define the number to factor
N = 15
print(f"We're going to factor N = {N}")

# Function to check if 'a' is coprime to N
def is_coprime(a, N):
    """
    Check if two numbers are coprime (have no common factors)
    
    Parameters:
    -----------
    a: int
        First number
    N: int
        Second number
        
    Returns:
    --------
    bool
        True if a and N are coprime, False otherwise
    """
    return gcd(a, N) == 1

# Function to display the pattern of modular exponentiation
def display_modular_pattern(a, N):
    """
    Display the pattern of a^x mod N for different values of x
    
    Parameters:
    -----------
    a: int
        Base for exponentiation
    N: int
        Modulus
        
    Returns:
    --------
    int
        The period of the function a^x mod N
    """
    print(f"\nPattern of a^x mod N where a={a}, N={N}:")
    print("x\ta^x\ta^x mod N")
    print("-" * 24)
    
    # Find the period by tracking values
    seen_values = {}
    period = None
    
    for x in range(2*N):  # Maximum period would be N
        mod_result = pow(a, x, N)
        print(f"{x}\t{a}^{x}\t{mod_result}")
        
        # Check if we've seen this value before
        if mod_result == 1 and x > 0:
            period = x
            break
    
    print(f"\nThe period r = {period}")
    return period

# Function to use the period to find factors
def find_factors_from_period(a, N, r):
    """
    Use the period to find factors of N
    
    Parameters:
    -----------
    a: int
        Base used in modular exponentiation
    N: int
        Number to factorize
    r: int
        Period of a^x mod N
        
    Returns:
    --------
    tuple
        (factor1, factor2) such that factor1 * factor2 = N
    """
    # Check if period is even
    if r % 2 != 0:
        return "Period is odd, cannot find factors using this method"
    
    # Check if a^(r/2) ≡ -1 (mod N)
    if pow(a, r//2, N) == N - 1:
        return "a^(r/2) ≡ -1 (mod N), try another value of a"
    
    # Calculate potential factors
    factor1 = gcd(pow(a, r//2) - 1, N)
    factor2 = gcd(pow(a, r//2) + 1, N)
    
    # Check if we found non-trivial factors
    if factor1 == 1 or factor1 == N:
        return factor2, N // factor2
    else:
        return factor1, N // factor1

### Task 1.2: Testing the Classical Components

Let's test our classical functions with a simple example:

In [ ]:
# Choose a random number a < N that is coprime to N
a = 7  # For demonstration, we'll choose 7

# Check if a is coprime to N and print the result
is_a_coprime = is_coprime(a, N)
print(f"Is a={a} coprime to N={N}? {is_a_coprime}")

# Display the pattern of a^x mod N and find the period
period = display_modular_pattern(a, N)

# Use the period to find factors and print the result
factors = find_factors_from_period(a, N, period)
print(f"\nFactors of {N} using period {period}: {factors}")

# Verify the factorization
if isinstance(factors, tuple):
    factor1, factor2 = factors
    print(f"Verification: {factor1} × {factor2} = {factor1 * factor2}")

## Exercise 2: Quantum Period Finding

The quantum part of Shor's algorithm is used to find the period of the function f(x) = a^x mod N. This is achieved using quantum phase estimation, which involves:

1. Creating a superposition of all possible inputs
2. Applying a controlled-U operation (where U|y⟩ = |a⋅y mod N⟩)
3. Using quantum Fourier transform to extract the period

Let's implement a simplified version of this quantum period-finding circuit:

In [ ]:
def create_quantum_period_finding_circuit(a, N, num_counting_qubits):
    """
    Create a quantum circuit for period finding
    
    Parameters:
    -----------
    a: int
        Base for modular exponentiation
    N: int
        Modulus
    num_counting_qubits: int
        Number of qubits to use for phase estimation
        
    Returns:
    --------
    QuantumCircuit
        The quantum circuit for period finding
    """
    # Calculate number of qubits needed for target register
    num_target_qubits = N.bit_length()
    
    # Total number of qubits
    total_qubits = num_counting_qubits + num_target_qubits
    
    # Create quantum circuit with counting and target registers
    qc = QuantumCircuit(total_qubits, num_counting_qubits)
    
    # Step 1: Apply Hadamard gates to counting qubits
    for i in range(num_counting_qubits):
        qc.h(i)
    
    # Step 2: Initialize target register to |1⟩
    qc.x(num_counting_qubits)  # Set the first target qubit to |1⟩
    
    # Step 3: Apply controlled-U operations
    # In a full implementation, this would be a modular exponentiation circuit
    # Here, we'll simulate it with controlled phase rotations for simplicity
    for i in range(num_counting_qubits):
        # Calculate phase rotation based on a^(2^i) mod N
        power = pow(a, 2**i, N)
        phase = (2 * np.pi * power) / N
        
        # Apply controlled phase rotation
        qc.cp(phase, i, num_counting_qubits)
    
    # Step 4: Apply inverse QFT to counting qubits
    # First create an inverse QFT circuit for the counting register
    iqft = QFT(num_counting_qubits, inverse=True)
    
    # Apply it to the counting qubits
    qc = qc.compose(iqft, qubits=range(num_counting_qubits))
    
    # Step 5: Add measurement to counting qubits
    qc.measure(range(num_counting_qubits), range(num_counting_qubits))
    
    return qc

# Create a simplified quantum period finding circuit
# For our example with N=15, a=7, using 4 counting qubits
circuit = create_quantum_period_finding_circuit(7, 15, 4)

# Display the circuit
print("Quantum Period Finding Circuit:")
display(circuit.draw(output='mpl', fold=-1))

### Task 2.2: Simulating the Quantum Circuit

Now, let's run the circuit and analyze the results:

In [ ]:
# Function to simulate the quantum circuit and process results
def run_period_finding(a, N, num_counting_qubits, shots=1024):
    """
    Run the quantum period finding circuit and process results
    
    Parameters:
    -----------
    a: int
        Base for modular exponentiation
    N: int
        Modulus
    num_counting_qubits: int
        Number of qubits for phase estimation
    shots: int
        Number of times to run the simulation
        
    Returns:
    --------
    dict
        Measurement counts from the simulation
    """
    # Create the circuit
    qc = create_quantum_period_finding_circuit(a, N, num_counting_qubits)
    
    # Run the circuit on the quantum simulator
    simulator = AerSimulator()
    job = simulator.run(qc, shots=shots)
    result = job.result()
    
    # Return the measurement results
    return result.get_counts()

# Run the quantum period finding for a=7, N=15
results = run_period_finding(7, 15, 4)

# Display the results as a histogram
plot_histogram(results)

# Process the results to estimate the period
print("\nProcessing quantum measurement results:")
for outcome, count in sorted(results.items(), key=lambda x: x[1], reverse=True):
    # Convert binary outcome to decimal
    decimal_value = int(outcome, 2)
    
    # Convert to phase (fraction of 2π)
    phase = decimal_value / (2**len(outcome))
    print(f"State: |{outcome}⟩, Count: {count}, Phase: {phase:.4f}")
    
    # Use continued fractions to find the period
    if count > 50:  # Only consider significant measurements
        fraction = Fraction(phase).limit_denominator(N)
        possible_period = fraction.denominator
        print(f"  Possible period from this measurement: {possible_period}")
        
        # Check if this is the correct period
        if possible_period > 0 and pow(a, possible_period, N) == 1:
            print(f"  This corresponds to a valid period!")

# For educational purposes, extract the best period estimate
best_outcome = max(results, key=results.get)
best_phase = int(best_outcome, 2) / (2**len(best_outcome))
best_period_estimate = Fraction(best_phase).limit_denominator(N).denominator

print(f"\nEstimated period from quantum circuit: {best_period_estimate}")

## Exercise 3: Complete Shor's Algorithm

Now, let's put everything together to implement a complete (albeit simplified) version of Shor's algorithm. We'll create a function that:

1. Performs preliminary checks (if N is even or a perfect power)
2. Selects a random a < N that is coprime to N
3. Uses quantum period finding to estimate the period
4. Uses the period to find the factors

In [ ]:
def run_shors_algorithm(N, num_counting_qubits=4, max_attempts=3):
    """
    Run Shor's algorithm to factor N
    
    Parameters:
    -----------
    N: int
        Number to factorize
    num_counting_qubits: int
        Number of qubits for phase estimation
    max_attempts: int
        Maximum number of attempts if period finding fails
        
    Returns:
    --------
    tuple
        (factor1, factor2) such that factor1 * factor2 = N
    """
    # Step 1: Check if N is even
    if N % 2 == 0:
        return 2, N//2
    
    # Step 2: Try different values of a until we find factors
    for attempt in range(max_attempts):
        # Choose a random number 1 < a < N
        a = np.random.randint(2, N)
        
        # Check if a is coprime to N
        if not is_coprime(a, N):
            # If a is not coprime to N, we've found a factor already
            factor = gcd(a, N)
            return factor, N // factor
        
        print(f"Attempt {attempt+1}: Using a = {a}")
        
        # Run quantum period finding
        results = run_period_finding(a, N, num_counting_qubits)
        
        # Process results to find the period
        # Extract the best measurement and convert to a period estimate
        best_outcome = max(results, key=results.get)
        phase = int(best_outcome, 2) / (2**len(best_outcome))
        period = Fraction(phase).limit_denominator(N).denominator
        
        print(f"  Estimated period: {period}")
        
        # Verify if this is a valid period
        if period > 0 and pow(a, period, N) == 1:
            print(f"  Confirmed valid period: {period}")
            
            # Use the period to find factors
            # Check if period is even and a^(r/2) ≠ -1 mod N
            if period % 2 == 0 and pow(a, period//2, N) != N-1:
                factor1 = gcd(pow(a, period//2) - 1, N)
                factor2 = gcd(pow(a, period//2) + 1, N)
                
                # Check if we found non-trivial factors
                if factor1 != 1 and factor1 != N:
                    return factor1, N // factor1
                if factor2 != 1 and factor2 != N:
                    return factor2, N // factor2
            
            print("  Period is valid but didn't yield factors. Trying again.")
    
    # If we've exhausted all attempts
    return "Failed to find factors after all attempts"

# Run Shor's algorithm on N=15
print("\nRunning Shor's algorithm to factor N =", N)
factors = run_shors_algorithm(N)
print(f"\nFactors of {N}: {factors}")

# Verify the factorization
if isinstance(factors, tuple):
    factor1, factor2 = factors
    print(f"Verification: {factor1} × {factor2} = {factor1 * factor2}")

## Exercise 4: Scaled-Down Implementation for Educational Purposes

Since implementing a full quantum circuit for modular exponentiation is quite complex, let's implement a simplified version of Shor's algorithm for educational purposes.

For N=15 and a=7, we can create a custom circuit that simulates the period-finding behavior:

In [ ]:
# Create a scaled-down circuit for period finding when N=15, a=7
def create_simplified_circuit():
    """
    Create a simplified circuit to demonstrate period finding for N=15, a=7
    
    Returns:
    --------
    QuantumCircuit
        A simplified quantum circuit
    """
    # For N=15, a=7, the period is 4
    # We'll create a circuit that gives measurements that would indicate period=4
    
    # Create a circuit with 3 qubits
    qc = QuantumCircuit(3, 3)
    
    # Apply Hadamard gates to all qubits
    qc.h(range(3))
    
    # Apply phase shifts to simulate the period=4 pattern
    # For period=4, we want to measure phases that are multiples of 1/4
    # Most frequently 0/4, 1/4, 2/4, 3/4
    qc.p(0, 0)        # Phase of 0/4
    qc.p(np.pi/2, 1)  # Phase of 1/4
    qc.p(np.pi, 2)    # Phase of 2/4
    
    # Apply inverse QFT
    qc.h(0)
    qc.cp(-np.pi/2, 0, 1)
    qc.h(1)
    qc.cp(-np.pi/4, 0, 2)
    qc.cp(-np.pi/2, 1, 2)
    qc.h(2)
    
    # Measure all qubits
    qc.measure(range(3), range(3))
    
    return qc

# Create and display the simplified circuit
simplified_circuit = create_simplified_circuit()
print("Simplified Circuit for Demonstrating Period=4:")
display(simplified_circuit.draw(output='mpl'))

# Run the simplified circuit and display results
simulator = AerSimulator()
job = simulator.run(simplified_circuit, shots=1024)
result = job.result()
counts = result.get_counts()

# Display the results
print("\nMeasurement results:")
for outcome, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
    binary_fraction = outcome  # Binary representation of the phase
    phase = int(outcome, 2) / 2**len(outcome)  # Convert to decimal phase
    print(f"State: |{outcome}⟩, Count: {count}, Phase: {phase:.4f}")
    
    # For educational purposes, interpret as fraction
    fraction = Fraction(phase).limit_denominator(15)
    print(f"  Interpreted as fraction: {fraction}")
    if fraction.denominator == 4:
        print(f"  This corresponds to period=4!")

# Visualize the results
plot_histogram(counts, title="Simplified Period Finding for Period=4")

## Reflection Questions

1. Why is Shor's algorithm considered a threat to RSA encryption? Explain the implications for cybersecurity.

2. The quantum part of Shor's algorithm provides an exponential speedup over classical algorithms. Why is finding the period of f(x) = a^x mod N so much faster on a quantum computer?

3. What are the main challenges in implementing Shor's algorithm on current quantum hardware? Consider the required number of qubits and circuit depth.

4. How many qubits would be needed to factor a 2048-bit RSA key using Shor's algorithm? Why is this currently infeasible?

## Reflection Answers

1. **Why is Shor's algorithm a threat to RSA encryption?**

   Shor's algorithm can efficiently factor large numbers, which directly attacks the mathematical foundation of RSA encryption. RSA's security relies on the computational difficulty of factoring the product of two large prime numbers. A classical computer would take billions of years to factor large RSA keys (2048 or 4096 bits), but a sufficiently powerful quantum computer using Shor's algorithm could potentially do it in hours or days. This would render most of our current public-key cryptography systems vulnerable, compromising secure communications across the internet, including e-commerce, banking, and confidential communications.

2. **Why is finding periods exponentially faster on quantum computers?**

   The exponential speedup comes from the quantum Fourier transform's ability to efficiently extract periodicities from functions. When we create a superposition of all possible inputs and apply the function, the quantum state encodes information about the entire function at once. The quantum Fourier transform can then extract the period in a single operation, essentially performing an exponential number of classical calculations simultaneously through quantum parallelism. Classically, we would need to evaluate the function for many different values and analyze the pattern, requiring O(N) operations, whereas the quantum approach requires only O((log N)²) operations.

3. **Main challenges in implementing Shor's algorithm on current hardware:**

   - **Qubit Count**: Factoring even modest numbers requires many high-quality qubits. For example, factoring a 1024-bit number might require thousands of logical qubits.
   - **Decoherence**: Quantum states are fragile and lose their quantum properties through interaction with the environment. The complex circuits needed for Shor's algorithm would need to maintain quantum coherence throughout the computation.
   - **Quantum Error Correction**: Current quantum computers have high error rates, requiring error correction that multiplies the physical qubit requirements by 10-1000×.
   - **Circuit Depth**: The modular exponentiation step requires deep circuits with many gates, increasing the probability of errors.
   - **Control Precision**: Implementing precise quantum gates with high fidelity is technically challenging.

4. **Qubits needed for 2048-bit RSA:**

   Factoring a 2048-bit number using Shor's algorithm would require approximately 2n to 3n qubits for the main register (where n=2048) plus additional qubits for calculation, totaling roughly 4000-8000 logical qubits. When quantum error correction is factored in, this could translate to millions of physical qubits with current error rates. 
   
   This is infeasible because:
   - The largest quantum computers today have only around 100-200 qubits
   - These qubits lack sufficient coherence time and gate fidelity for such complex algorithms
   - Current error rates would require prohibitively large numbers of physical qubits
   - The complex quantum circuits required for modular exponentiation of large numbers have not been successfully implemented at scale

## Conclusion

In this lab, you've implemented and analyzed Shor's algorithm, one of the most powerful quantum algorithms with profound implications for cryptography. You've learned:

1. How the factoring problem relates to finding periods of modular functions
2. How quantum computers can find these periods exponentially faster than classical computers
3. How to implement the quantum period-finding subroutine
4. How to use the period to determine factors

While a full implementation of Shor's algorithm for large numbers is beyond the capabilities of current quantum computers, understanding the algorithm provides valuable insight into the potential power of quantum computing and its implications for the future of cryptography.